In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('FortLauderdaleCoastalBrowardCondoMap - BrowardCondoPipelineMap.csv',dtype='str')

## Data clean

In [3]:
df = df.dropna(subset='Address')

In [4]:
df = df.drop(columns=['Developer Checked','Emailed PR'])

In [5]:
df['Full_Address'] = df['Address'] + ' ' + df['City'] + ' , FL'

## Geocode

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

## Geocoded

In [8]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [9]:
df['Full_Address'] = df['Address'] + ' ' + df['City'] + ', FL'

In [10]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

In [11]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Link Formatter

In [12]:
df['Story Link'] = 'Click here for story'

df["short_description"] = df["Story Link"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Story Link'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["Story Link"] = '<a href="' + df["URL"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]


In [13]:
df.columns

Index(['Address', 'Project name', 'Developer', 'Description', 'City', 'URL',
       'Unit County', 'Unnamed: 9', 'Full_Address', 'geocoded', 'lat', 'lon',
       'Story Link', 'short_description', 'remaining_desc'],
      dtype='object')

In [25]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap
import html

# Handle missing data and ensure all fields are strings
df["Address"] = df["Address"].fillna("N/A").astype(str)
df["Project name"] = df["Project name"].fillna("N/A").astype(str)
df["Developer"] = df["Developer"].fillna("N/A").astype(str)
df["Full_Address"] = df["Full_Address"].fillna("N/A").astype(str)
df["Description"] = df["Description"].fillna("N/A").astype(str)
df["URL"] = df["URL"].fillna("#").astype(str)

# Function to sanitize text to prevent HTML injection
def sanitize(text):
    return html.escape(text, quote=True)

# Apply sanitization
df["Address"] = df["Address"].apply(sanitize)
df["Project name"] = df["Project name"].apply(sanitize)
df["Developer"] = df["Developer"].apply(sanitize)
df["Full_Address"] = df["Full_Address"].apply(sanitize)
df["Description"] = df["Description"].apply(sanitize)
df["URL"] = df["URL"].apply(sanitize)

# Shorten the Description to create 'short_description'
df["short_description"] = df["Description"].apply(
    lambda s: textwrap.shorten(s, width=20, placeholder="...")
)

# Create 'remaining_desc' by removing 'short_description' from 'Description'
df['remaining_desc'] = df.apply(
    lambda row: row['Description'].replace(row['short_description'], '', 1), axis=1
)

# Create 'description_link' with the sanitized data
df["description_link"] = (
    f'<a href="{df["URL"]}" target="_blank" rel="noopener noreferrer">' +
    df["short_description"] +
    "</a>" +
    df["remaining_desc"]
)

# Define the popup HTML function
def tooltip_html(row_index, max_chars=30):
    """Generate HTML content for the tooltip with line breaks for long text."""
    tooltip_content = '<div class="popup-content">'

    # Dynamically add data for each column
    for column in df.columns:
        value = df.at[row_index, column]
        value_str = str(value).strip()
        if pd.notnull(value) and value_str != "" and column in ['Address', 'Project name', 'Developer', 'Description']:

            # Check if the string exceeds the maximum character length
            if len(value_str) > max_chars and column == "Site Address":
                # Insert a <br> tag every max_chars characters
                value_str = '<br>'.join(value_str[i:i+max_chars] for i in range(0, len(value_str), max_chars))

            tooltip_content += f'<strong>{column}:</strong> {value_str}<br>'

    tooltip_content += '</div>'
    return tooltip_content

# Initialize the map centered around the mean latitude and longitude
map_center = [df['lat'].mean(), df['lon'].mean()]

m = folium.Map(location=map_center, zoom_start=7, tiles=None)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr = 'Mapbox',
    name = 'Streets',
    overlay = False,
    control = True,
    show = False,
    min_zoom = 11,
    max_zoom = 30
    ).add_to(m)


# Create and add the title to the map
title_html = '''
<h3 align="center" style="font-size:16px"><b>Fort Lauderdale Condo Pipeline</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
# marker_cluster = MarkerCluster().add_to(m)


# Loop through the DataFrame to create markers
for i in range(len(df)):
    # Create tooltip HTML content
    tooltip_content = tooltip_html(i)
    
    # Create a custom icon
    icon = folium.Icon(color='red')
    
    # Create a marker with a tooltip
    marker = folium.CircleMarker(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        tooltip=tooltip_content,
        color='red',
        fill=True,
        radius=7,
        icon=folium.CircleMarker(color='red')
    ).add_to(m)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m


In [26]:
m.save("index.html")

In [27]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/FortLauderdalePipelineMap_01_14_24
